In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import norm
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import pickle
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
def selectkbest(indep_X,dep_Y,n):
    test=SelectKBest(score_func=chi2, k=n)
    fit1=test.fit(indep_X,dep_Y)
    selectK_features=fit1.transform(indep_X)
    selected_col=indep_X.columns[fit1.get_support(indices=True)]
    return selectK_features, selected_col

def scaler(indep_X,dep_Y):
    sc = StandardScaler()
    scy=StandardScaler()
    indep_X = sc.fit_transform(indep_X)
    dep_Y = scy.fit_transform(dep_Y)
    return indep_X,dep_Y

def r2_prediction(grid,indep_X,dep_Y):
    y_pred = grid.predict(indep_X)
    r2=r2_score(dep_Y,y_pred)
    return r2

def Linear(indep_X,dep_Y):
    param_grid={'fit_intercept':[True,False],'copy_X':[True,False],'positive':[False,True]}
    grid=GridSearchCV(LinearRegression(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2

def svm(indep_X,dep_Y):
    param_grid={'kernel':['linear', 'poly', 'rbf', 'sigmoid'],'C':[10,100,1000,2000,3000,5000],'gamma':['auto','scale']}
    grid=GridSearchCV(SVR(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2 

def Decision(indep_X,dep_Y):
    param_grid={'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],'splitter':['best', 'random'],'max_features':['sqrt', 'log2']}
    grid=GridSearchCV(DecisionTreeRegressor(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2 

def random(indep_X,dep_Y):
    param_grid={'n_estimators':[10,50,100],'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],'max_features':['sqrt', 'log2']}
    grid=GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose=3,n_jobs=-1)
    grid.fit(indep_X,dep_Y)
    r2=r2_prediction(grid,indep_X,dep_Y)
    return  r2  

def selectk_regression(acclin,accsvml,accsvmnl,accdes,accrf): 
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Linear','SVM','Decision','Random'])
    for number,idex in enumerate(dataframe.index):        
        dataframe.loc[idex,'Linear']=acclin[number]       
        dataframe.loc[idex,'SVM']=accsvm[number]
        dataframe.loc[idex,'Decision']=accdes[number]
        dataframe.loc[idex,'Random']=accrf[number]
    return dataframe

In [3]:
dataset=pd.read_csv('sales_prep.csv',index_col=None)
df=dataset

dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

indep_X=dataset.drop('Sales_Revenue',axis=1)
dep_Y=dataset['Sales_Revenue']

In [4]:
kbest,selected=selectkbest(indep_X,dep_Y,3)

indep_X,dep_Y=scaler(indep_X,dep_Y)

acclin=[]
accsvm=[]
accdes=[]
accrf=[]

r2_lin=Linear(indep_X,dep_Y)
acclin.append(r2_lin)
    
r2_svm=svm(indep_X,dep_Y)
accsvm.append(r2_svm)
    
r2_d=Decision(indep_X,dep_Y)
accdes.append(r2_d)
    
r2_r=random(indep_X,dep_Y)
accrf.append(r2_r)

result=selectk_regression(acclin,accsvm,accdes,accrf)

ValueError: Unknown label type: (array([149827.17043977, 116907.7441383 ,  42876.1678113 , 120987.62157708,
       152300.67507458,  31988.97555907, 164296.80846844, 113301.47490077,
         5833.66442291, 106356.80483591,  82155.94096668,  62807.06970155,
       155395.15788399, 180402.47731348, 153717.3524223 , 170120.65341022,
       182212.28392618, 182962.71571841,  27354.43788437,  93995.170333  ,
         5055.86250662, 103760.20401722, 185227.56719704,  43824.94662764,
       129758.39260948, 130653.91405149,  36269.02701412,  51512.8506669 ,
        95094.24787225,  41700.8284086 , 119073.82581228,  99485.63754934,
       106243.64480991,  31221.17280001, 105864.10596054,  14648.24725556,
       176899.38311228,  71933.68970709, 171985.77834296, 141822.99190524,
       154403.65491295,  66336.17643509,  49410.06826764,  84314.40799902,
         9724.0431705 , 184157.35698933,  80467.78186339, 197768.5379445 ,
       153482.63705745,  20700.42691087, 133736.7141793 ,  57813.63